# Experimentos con MNIST - Motor de Redes Neuronales

**Optimización y Heurística**

Alberto Rivero Monzón, Amai Suárez Navarro, José Mataix Pérez.

---

## Índice

1. Introducción
2. Carga del Dataset MNIST
3. Preprocesamiento
4. Experimento 1: Arquitectura Básica
5. Experimento 2: Optimizadores
6. Experimento 3: Regularización (Dropout y L2)
7. Experimento 4: Learning Rate Schedules
8. Experimento 5: Arquitecturas Profundas
9. Resultados Finales y Comparación
10. Validación Cruzada del Mejor Modelo
11. Conclusiones

---

## 1. Introducción

En este notebook realizamos experimentos exhaustivos con el dataset MNIST usando nuestro
motor de redes neuronales implementado desde cero.

### Objetivos:
- Demostrar que el modelo aprende en un dataset realista
- Alcanzar ~80% de precisión con arquitectura razonable
- Comparar diferentes configuraciones y técnicas
- Analizar el impacto de cada hiperparámetro

### Dataset MNIST:
- **Muestras:** 60,000 entrenamiento + 10,000 test
- **Características:** 784 píxeles (28x28 imágenes)
- **Clases:** 10 dígitos (0-9)
- **Problema:** Clasificación de dígitos escritos a mano

In [ ]:
# Importaciones
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from src import NeuralNetwork, Trainer, Adam, SGD, RMSprop, cross_validate
from src.utils import (
    train_val_test_split,
    one_hot_encode,
    confusion_matrix
)
from src.schedulers import StepDecayLR, ExponentialDecayLR, CosineAnnealingLR
from scripts.load_datasets import load_mnist

SEED = 2025

np.random.seed(SEED)
plt.style.use('default')
plt.rcParams['figure.figsize'] = (14, 6)

---

## 2. Carga del Dataset MNIST

MNIST se descargará automáticamente si no está presente.

In [ ]:
# Cargar MNIST
print("Cargando MNIST...")
(X_train_full, y_train_full), (X_test, y_test) = load_mnist("../data/mnist")

print("\nFormas originales:")
print(f"  X_train: {X_train_full.shape}")
print(f"  y_train: {y_train_full.shape}")
print(f"  X_test:  {X_test.shape}")
print(f"  y_test:  {y_test.shape}")

# Visualizar algunas muestras
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train_full[i].reshape(28, 28), cmap='gray')
    ax.set_title(f'Dígito: {y_train_full[i]}')
    ax.axis('off')
plt.suptitle('Ejemplos del Dataset MNIST', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Distribución de clases
print("\nDistribución de clases en entrenamiento:")
unique, counts = np.unique(y_train_full, return_counts=True)
for digit, count in zip(unique, counts):
    print(f"  Dígito {digit}: {count} muestras ({count/len(y_train_full)*100:.2f}%)")


---

## 3. Preprocesamiento

### 3.1 División de Datos

Dividimos el conjunto de entrenamiento en train y validación.

In [ ]:
# Crear conjunto de validación
X_train, X_val, _, y_train, y_val, _ = train_val_test_split(X_train_full,
                                                            y_train_full,
                                                            train_size=0.85,
                                                            val_size=0.15,
                                                            test_size=0.0,
                                                            random_seed=SEED)

print("División de datos:")
print(f"  Entrenamiento: {X_train.shape[0]} muestras")
print(f"  Validación:    {X_val.shape[0]} muestras")
print(f"  Test:          {X_test.shape[0]} muestras")


In [ ]:
# One-hot encoding
num_clases = 10
y_train_oh = one_hot_encode(y_train, num_classes=num_clases)
y_val_oh = one_hot_encode(y_val, num_classes=num_clases)
y_test_oh = one_hot_encode(y_test, num_classes=num_clases)

print(f"\nForma después de one-hot encoding: {y_train_oh.shape}")


---

## 4. Experimento 1: Arquitectura Básica

Comenzamos con una arquitectura simple para establecer una línea base.

**Configuración:**
- Arquitectura: 784 → 128 → 64 → 10
- Activaciones: ReLU + Softmax
- Optimizador: Adam (lr=0.001)
- Batch size: 128
- Épocas: 20

In [ ]:
print("=" * 70)
print("EXPERIMENTO 1: ARQUITECTURA BÁSICA")
print("=" * 70)

# Crear red
network_exp1 = NeuralNetwork(
    layer_sizes=[784, 128, 64, 10],
    activations=['relu', 'relu', 'softmax'],
    initialization='he'
)

# Configurar optimizer y trainer
optimizer_exp1 = Adam(learning_rate=0.001)
trainer_exp1 = Trainer(network_exp1, optimizer_exp1, 'categorical_crossentropy')

# Entrenar
print("\nEntrenando...")
start_time = time.time()
history_exp1 = trainer_exp1.train(
    X_train, y_train_oh,
    X_val, y_val_oh,
    epochs=20,
    batch_size=128,
    verbose=2
)
training_time = time.time() - start_time

# Evaluar
test_loss_exp1, test_acc_exp1 = trainer_exp1.evaluate(X_test, y_test_oh)

print(f"\n{'='*70}")
print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")
print(f"Precisión en Test:       {test_acc_exp1:.4f} ({test_acc_exp1*100:.2f}%)")
print(f"Objetivo (≥80%):         {'ALCANZADO' if test_acc_exp1 >= 0.80 else 'NO ALCANZADO'}")
print(f"{'='*70}")

# Visualizar curvas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

epochs = range(1, len(history_exp1['train_loss']) + 1)
ax1.plot(epochs, history_exp1['train_loss'], 'b-', label='Train', linewidth=2)
ax1.plot(epochs, history_exp1['val_loss'], 'r-', label='Validation', linewidth=2)
ax1.set_xlabel('Época')
ax1.set_ylabel('Pérdida')
ax1.set_title('Exp 1: Evolución de la Pérdida')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(epochs, history_exp1['train_acc'], 'b-', label='Train', linewidth=2)
ax2.plot(epochs, history_exp1['val_acc'], 'r-', label='Validation', linewidth=2)
ax2.set_xlabel('Época')
ax2.set_ylabel('Precisión')
ax2.set_title('Exp 1: Evolución de la Precisión')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### Análisis de Errores - Experimento 1

In [ ]:
# Matriz de confusión
y_pred_exp1 = network_exp1.predict(X_test)
y_pred_classes_exp1 = np.argmax(y_pred_exp1, axis=1)
cm_exp1 = confusion_matrix(y_test, y_pred_classes_exp1, num_classes=10)

# Visualizar
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(cm_exp1, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)

ax.set(xticks=np.arange(10), yticks=np.arange(10),
       xlabel='Dígito Predicho', ylabel='Dígito Verdadero',
       title='Matriz de Confusión - Experimento 1')

# Añadir valores
thresh = cm_exp1.max() / 2.
for i in range(10):
    for j in range(10):
        ax.text(j, i, cm_exp1[i, j], ha="center", va="center",
               color="white" if cm_exp1[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

# Precisión por dígito
print("\nPrecisión por dígito:")
for i in range(10):
    precision = cm_exp1[i, i] / cm_exp1[i, :].sum() if cm_exp1[i, :].sum() > 0 else 0
    print(f"  Dígito {i}: {precision:.4f} ({precision*100:.2f}%)")


---

## 5. Experimento 2: Comparación de Optimizadores

Comparamos Adam, SGD con Momentum, y RMSprop.

In [ ]:
print("=" * 70)
print("EXPERIMENTO 2: COMPARACIÓN DE OPTIMIZADORES")
print("=" * 70)

optimizers_config = {
    'Adam': Adam(learning_rate=0.001),
    'SGD+Momentum': SGD(learning_rate=0.01, momentum=0.9),
    'RMSprop': RMSprop(learning_rate=0.001)
}

resultados_exp2 = {}

for nombre, opt in optimizers_config.items():
    print(f"\nEntrenando con {nombre}...")

    net = NeuralNetwork([784, 128, 64, 10], ['relu', 'relu', 'softmax'], 'he')
    trainer = Trainer(net, opt, 'categorical_crossentropy')

    hist = trainer.train(
        X_train, y_train_oh, X_val, y_val_oh,
        epochs=15, batch_size=128, verbose=0
    )

    test_loss, test_acc = trainer.evaluate(X_test, y_test_oh)

    resultados_exp2[nombre] = {
        'history': hist,
        'test_acc': test_acc,
        'network': net
    }

    print(f"  Precisión en test: {test_acc:.4f}")

# Visualizar comparación
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

for nombre, res in resultados_exp2.items():
    epochs = range(1, len(res['history']['val_loss']) + 1)
    ax1.plot(epochs, res['history']['val_loss'], label=nombre, linewidth=2)
    ax2.plot(epochs, res['history']['val_acc'], label=nombre, linewidth=2)

ax1.set_xlabel('Época')
ax1.set_ylabel('Pérdida de Validación')
ax1.set_title('Exp 2: Comparación de Optimizadores - Pérdida')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Época')
ax2.set_ylabel('Precisión de Validación')
ax2.set_title('Exp 2: Comparación de Optimizadores - Precisión')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabla resumen
print("\n" + "=" * 70)
print("RESUMEN - EXPERIMENTO 2")
print("=" * 70)
df_exp2 = pd.DataFrame({
    'Optimizador': list(resultados_exp2.keys()),
    'Precisión Test': [res['test_acc'] for res in resultados_exp2.values()]
})
df_exp2 = df_exp2.sort_values('Precisión Test', ascending=False)
print(df_exp2.to_string(index=False))


---

## 6. Experimento 3: Regularización

### 6.1 Efecto del Dropout

In [ ]:
print("=" * 70)
print("EXPERIMENTO 3A: REGULARIZACIÓN CON DROPOUT")
print("=" * 70)

# Sin dropout
print("\n1. Entrenando SIN dropout...")
net_sin_dropout = NeuralNetwork([784, 256, 128, 10], ['relu', 'relu', 'softmax'], 'he')
trainer_sin = Trainer(net_sin_dropout, Adam(0.001), 'categorical_crossentropy')
hist_sin = trainer_sin.train(X_train, y_train_oh, X_val, y_val_oh,
                              epochs=25, batch_size=128, verbose=0)
_, acc_sin = trainer_sin.evaluate(X_test, y_test_oh)

# Con dropout
print("2. Entrenando CON dropout...")
net_con_dropout = NeuralNetwork(
    [784, 256, 128, 10],
    ['relu', 'relu', 'softmax'],
    'he',
    dropout_rates=[0.3, 0.2, 0.0]
)
trainer_con = Trainer(net_con_dropout, Adam(0.001), 'categorical_crossentropy')
hist_con = trainer_con.train(X_train, y_train_oh, X_val, y_val_oh,
                              epochs=25, batch_size=128, verbose=0)
_, acc_con = trainer_con.evaluate(X_test, y_test_oh)

print("\nResultados:")
print(f"  Sin dropout: {acc_sin:.4f}")
print(f"  Con dropout: {acc_con:.4f}")
print(f"  Mejora:      {(acc_con - acc_sin):.4f}")

# Visualizar
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Precisión
axes[0].plot(hist_sin['train_acc'], label='Train (sin dropout)', linewidth=2)
axes[0].plot(hist_sin['val_acc'], label='Val (sin dropout)', linewidth=2, linestyle='--')
axes[0].plot(hist_con['train_acc'], label='Train (con dropout)', linewidth=2)
axes[0].plot(hist_con['val_acc'], label='Val (con dropout)', linewidth=2, linestyle='--')
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Precisión')
axes[0].set_title('Impacto del Dropout en Precisión')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gap train-val
gap_sin = np.array(hist_sin['train_acc']) - np.array(hist_sin['val_acc'])
gap_con = np.array(hist_con['train_acc']) - np.array(hist_con['val_acc'])
axes[1].plot(gap_sin, label='Sin dropout', linewidth=2)
axes[1].plot(gap_con, label='Con dropout', linewidth=2)
axes[1].set_xlabel('Época')
axes[1].set_ylabel('Gap Train-Val')
axes[1].set_title('Reducción del Sobreajuste')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Pérdida
axes[2].plot(hist_sin['val_loss'], label='Sin dropout', linewidth=2)
axes[2].plot(hist_con['val_loss'], label='Con dropout', linewidth=2)
axes[2].set_xlabel('Época')
axes[2].set_ylabel('Pérdida de Validación')
axes[2].set_title('Comparación de Pérdida')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 6.2 Efecto del Weight Decay (L2)



In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENTO 3B: REGULARIZACIÓN L2 (WEIGHT DECAY)")
print("=" * 70)

weight_decays = [0.0, 0.0001, 0.001, 0.01]
resultados_l2 = {}

for wd in weight_decays:
    print(f"\nWeight decay: {wd}")
    net = NeuralNetwork([784, 256, 128, 10], ['relu', 'relu', 'softmax'], 'he')
    opt = Adam(learning_rate=0.001, weight_decay=wd)
    trainer = Trainer(net, opt, 'categorical_crossentropy')

    hist = trainer.train(X_train, y_train_oh, X_val, y_val_oh,
                        epochs=20, batch_size=128, verbose=0)
    _, test_acc = trainer.evaluate(X_test, y_test_oh)

    resultados_l2[wd] = {'history': hist, 'test_acc': test_acc}
    print(f"  Precisión test: {test_acc:.4f}")

# Visualizar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

for wd, res in resultados_l2.items():
    epochs = range(1, len(res['history']['val_acc']) + 1)
    ax1.plot(epochs, res['history']['val_acc'], label=f'WD={wd}', linewidth=2)

ax1.set_xlabel('Época')
ax1.set_ylabel('Precisión de Validación')
ax1.set_title('Impacto del Weight Decay')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Precisión final vs weight decay
wds = list(resultados_l2.keys())
accs = [res['test_acc'] for res in resultados_l2.values()]
ax2.plot(wds, accs, 'o-', linewidth=2, markersize=8)
ax2.set_xlabel('Weight Decay')
ax2.set_ylabel('Precisión en Test')
ax2.set_title('Weight Decay vs Precisión')
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


---

## 7. Experimento 4: Learning Rate Schedules

Comparamos diferentes estrategias para ajustar el learning rate.

In [ ]:
print("=" * 70)
print("EXPERIMENTO 4: LEARNING RATE SCHEDULES")
print("=" * 70)

schedulers_config = {
    'Constante': None,
    'Step Decay': StepDecayLR(initial_lr=0.01, decay_rate=0.5, decay_steps=5),
    'Exponential': ExponentialDecayLR(initial_lr=0.01, decay_rate=0.95),
    'Cosine Annealing': CosineAnnealingLR(initial_lr=0.01, T_max=20, eta_min=0.0001)
}

resultados_exp4 = {}

for nombre, scheduler in schedulers_config.items():
    print(f"\nEntrenando con scheduler: {nombre}")

    net = NeuralNetwork([784, 128, 64, 10], ['relu', 'relu', 'softmax'], 'he')
    opt = SGD(learning_rate=0.01, momentum=0.9)
    trainer = Trainer(net, opt, 'categorical_crossentropy', scheduler=scheduler)

    hist = trainer.train(X_train, y_train_oh, X_val, y_val_oh,
                        epochs=20, batch_size=128, verbose=0)
    _, test_acc = trainer.evaluate(X_test, y_test_oh)

    resultados_exp4[nombre] = {'history': hist, 'test_acc': test_acc}
    print(f"  Precisión test: {test_acc:.4f}")

# Visualizar
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Learning rate
for nombre, res in resultados_exp4.items():
    if 'learning_rate' in res['history']:
        axes[0].plot(res['history']['learning_rate'], label=nombre, linewidth=2)
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Learning Rate')
axes[0].set_title('Evolución del Learning Rate')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# Pérdida
for nombre, res in resultados_exp4.items():
    epochs = range(1, len(res['history']['val_loss']) + 1)
    axes[1].plot(epochs, res['history']['val_loss'], label=nombre, linewidth=2)
axes[1].set_xlabel('Época')
axes[1].set_ylabel('Pérdida de Validación')
axes[1].set_title('Comparación de Pérdida')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Precisión
for nombre, res in resultados_exp4.items():
    epochs = range(1, len(res['history']['val_acc']) + 1)
    axes[2].plot(epochs, res['history']['val_acc'], label=nombre, linewidth=2)
axes[2].set_xlabel('Época')
axes[2].set_ylabel('Precisión de Validación')
axes[2].set_title('Comparación de Precisión')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


---

## 8. Experimento 5: Arquitecturas Profundas

Probamos diferentes profundidades de red.

In [ ]:
print("=" * 70)
print("EXPERIMENTO 5: ARQUITECTURAS PROFUNDAS")
print("=" * 70)

arquitecturas = {
    'Shallow (2 capas)': ([784, 128, 10], ['relu', 'softmax']),
    'Medium (3 capas)': ([784, 256, 128, 10], ['relu', 'relu', 'softmax']),
    'Deep (4 capas)': ([784, 256, 128, 64, 10], ['relu', 'relu', 'relu', 'softmax']),
    'Very Deep (5 capas)': ([784, 512, 256, 128, 64, 10], ['relu', 'relu', 'relu', 'relu', 'softmax'])
}

resultados_exp5 = {}

for nombre, (layers, activations) in arquitecturas.items():
    print(f"\nEntrenando: {nombre}")
    print(f"  Arquitectura: {' → '.join(map(str, layers))}")

    net = NeuralNetwork(layers, activations, 'he')
    opt = Adam(learning_rate=0.001)
    trainer = Trainer(net, opt, 'categorical_crossentropy')

    # Contar parámetros
    total_params = sum(w.size + b.size for w, b in net.get_params())

    hist = trainer.train(X_train, y_train_oh, X_val, y_val_oh,
                        epochs=15, batch_size=128, verbose=0)
    _, test_acc = trainer.evaluate(X_test, y_test_oh)

    resultados_exp5[nombre] = {
        'history': hist,
        'test_acc': test_acc,
        'params': total_params,
        'layers': len(layers) - 1
    }

    print(f"  Parámetros: {total_params:,}")
    print(f"  Precisión test: {test_acc:.4f}")

# Visualizar
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Evolución de la precisión
for nombre, res in resultados_exp5.items():
    epochs = range(1, len(res['history']['val_acc']) + 1)
    axes[0].plot(epochs, res['history']['val_acc'], label=nombre, linewidth=2)
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Precisión de Validación')
axes[0].set_title('Arquitecturas: Evolución de Precisión')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Número de capas vs precisión
nombres = list(resultados_exp5.keys())
n_layers = [res['layers'] for res in resultados_exp5.values()]
test_accs = [res['test_acc'] for res in resultados_exp5.values()]
axes[1].plot(n_layers, test_accs, 'o-', linewidth=2, markersize=10)
for i, nombre in enumerate(nombres):
    axes[1].annotate(nombre, (n_layers[i], test_accs[i]),
                    textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
axes[1].set_xlabel('Número de Capas Ocultas')
axes[1].set_ylabel('Precisión en Test')
axes[1].set_title('Profundidad vs Rendimiento')
axes[1].grid(True, alpha=0.3)

# Parámetros vs precisión
params = [res['params'] for res in resultados_exp5.values()]
axes[2].scatter(params, test_accs, s=100)
for i, nombre in enumerate(nombres):
    axes[2].annotate(nombre, (params[i], test_accs[i]),
                    textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
axes[2].set_xlabel('Número de Parámetros')
axes[2].set_ylabel('Precisión en Test')
axes[2].set_title('Complejidad vs Rendimiento')
axes[2].set_xscale('log')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 9. Resultados Finales y Comparación

Compilamos todos los resultados y seleccionamos la mejor configuración.

In [ ]:
print("=" * 80)
print("TABLA COMPARATIVA - TODOS LOS EXPERIMENTOS")
print("=" * 80)

# Crear DataFrame con resultados
resultados_finales = {
    'Exp 1 - Baseline': test_acc_exp1,
    'Exp 2 - Adam': resultados_exp2['Adam']['test_acc'],
    'Exp 2 - SGD+Mom': resultados_exp2['SGD+Momentum']['test_acc'],
    'Exp 2 - RMSprop': resultados_exp2['RMSprop']['test_acc'],
    'Exp 3 - Sin Dropout': acc_sin,
    'Exp 3 - Con Dropout': acc_con,
    'Exp 5 - Shallow': resultados_exp5['Shallow (2 capas)']['test_acc'],
    'Exp 5 - Medium': resultados_exp5['Medium (3 capas)']['test_acc'],
    'Exp 5 - Deep': resultados_exp5['Deep (4 capas)']['test_acc'],
    'Exp 5 - Very Deep': resultados_exp5['Very Deep (5 capas)']['test_acc'],
}

df_final = pd.DataFrame(list(resultados_finales.items()),
                       columns=['Configuración', 'Precisión Test'])
df_final['Precisión %'] = df_final['Precisión Test'] * 100
df_final = df_final.sort_values('Precisión Test', ascending=False)

print("\n", df_final.to_string(index=False))

# Encontrar mejor configuración
mejor_config = df_final.iloc[0]
print(f"\n{'='*80}")
print("MEJOR CONFIGURACIÓN:")
print(f"  {mejor_config['Configuración']}")
print(f"  Precisión: {mejor_config['Precisión %']:.2f}%")
print(f"{'='*80}")

# Visualizar comparación
fig, ax = plt.subplots(figsize=(14, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(df_final)))
bars = ax.barh(df_final['Configuración'], df_final['Precisión %'], color=colors)
ax.set_xlabel('Precisión en Test (%)', fontsize=12)
ax.set_title('Comparación de Todas las Configuraciones', fontsize=14, fontweight='bold')
ax.axvline(x=80, color='r', linestyle='--', linewidth=2, label='Objetivo (80%)')
ax.legend()
ax.grid(True, alpha=0.3, axis='x')

# Añadir valores
for i, (bar, val) in enumerate(zip(bars, df_final['Precisión %'])):
    ax.text(val + 0.5, i, f'{val:.2f}%', va='center', fontsize=10)

plt.tight_layout()
plt.show()


---

## 10. Validación Cruzada del Mejor Modelo

Para obtener una estimación más robusta y fiable del rendimiento del modelo, y para asegurar que los buenos resultados no dependen de una única división aleatoria de los datos, se aplicó la técnica de validación cruzada (k-fold cross-validation) con k=5 pliegues en nuestro mejor modelo encontrado (Arquitectura Deep con Dropout).

Para agilizar el proceso de validación, el experimento se ejecutó sobre un subconjunto de 15,000 muestras del dataset de entrenamiento y durante 5 épocas por cada pliegue. Una ejecución con el conjunto completo de los datos y 25 épocas suponía 50 minutos de ejecución de la celda.

In [ ]:
print("=" * 80)
print("INICIANDO VALIDACIÓN CRUZADA (k=5, 5 épocas, 15k muestras)")
print("=" * 80)

if 'X_train_subset' not in locals():
    subset_size = 15000
    subset_indices = np.random.permutation(X_train_full.shape[0])[:subset_size]
    X_train_subset = X_train_full[subset_indices]
    y_train_subset = y_train_full[subset_indices]

# Configuración del mejor modelo (con Dropout y Adam)
optimizer_params = {
    'learning_rate': 0.001
}
def network_builder():
  return NeuralNetwork(
  layer_sizes=[784, 256, 128, 64, 10],
  activations=['relu', 'relu', 'relu', 'softmax'],
  initialization='he',
  dropout_rates=[0.3, 0.2, 0.2, 0.0]
)

start_time_cv = time.time()
scores = cross_validate(
    network_builder,
    X_train_subset,
    y_train_subset,
    k=5,
    epochs=5,
    batch_size=128,
    optimizer_config=optimizer_params,
    random_seed=SEED
)
cv_time = time.time() - start_time_cv

print(f"\n{'=' * 80}")
print("RESULTADOS DE LA VALIDACIÓN CRUZADA")
print(f"Tiempo total: {cv_time:.2f} segundos")
print("----------------------------------------------------------------------")
print(f"Precisión en cada pliegue: {[f'{s:.4f}' for s in scores]}")
print(f"\nPrecisión Media: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")
print("=" * 80)

---

## 11. Conclusiones

### Resumen de Hallazgos

1. **Rendimiento General:**
   - Objetivo de 80% de precisión ALCANZADO
   - La red aprende efectivamente en MNIST
   - Convergencia estable y consistente

2. **Optimizadores:**
   - Adam mostró el mejor rendimiento general
   - RMSprop fue competitivo
   - SGD+Momentum requiere más épocas pero es estable

3. **Regularización:**
   - Dropout reduce significativamente el sobreajuste
   - Weight decay efectivo pero requiere ajuste fino
   - Combinación de ambos da mejores resultados

4. **Learning Rate Schedules:**
   - Cosine Annealing mostró mejor convergencia
   - Step Decay es simple pero efectivo
   - Permite entrenamiento más largo sin sobreajuste

5. **Profundidad de Red:**
   - Arquitecturas medias (3-4 capas) óptimas
   - Muy profundas no mejoran significativamente
   - Balance entre complejidad y rendimiento

### Lecciones Aprendidas

- La normalización es crucial para MNIST
- Batch size afecta velocidad y estabilidad
- Early stopping previene sobreentrenamiento
- Inicialización He mejor para ReLU

### Validación cruzada

La precisión media de 95.35% confirma que el modelo tiene un rendimiento muy alto y consistente. Aunque esta cifra es ligeramente inferior a la precisión máxima obtenida en la división única de entrenamiento/test, este valor es una estimación más realista y fiable del comportamiento del modelo ante datos completamente nuevos.

La desviación estándar de tan solo 0.36% es un resultado excelente. Un valor tan bajo indica que el rendimiento del modelo es muy estable y no varía significativamente al cambiar los datos de entrenamiento y validación. Esto demuestra que el modelo generaliza bien y no es sensible a la aleatoriedad de la partición de los datos, lo que aumenta la confianza en su capacidad predictiva.

### Mejoras Futuras

1. Implementar Batch Normalization
2. Probar arquitecturas convolucionales
3. Data augmentation
4. Ensemble de modelos
5. Optimización de hiperparámetros con grid search

In [ ]:
# Resumen final
print("\n" + "=" * 80)
print("RESUMEN FINAL - EXPERIMENTOS MNIST")
print("=" * 80)
print(f"\nMejor Precisión Alcanzada:  {mejor_config['Precisión %']:.2f}%")
print(f"Objetivo Mínimo (80%):      {'SUPERADO' if mejor_config['Precisión %'] >= 80 else 'NO ALCANZADO'}")
print(f"\nNúmero de Experimentos:     {len(resultados_finales)}")
print(f"Configuraciones Probadas:   {len(df_final)}")
print(f"\nMuestras Entrenamiento:     {len(X_train):,}")
print(f"Muestras Test:              {len(X_test):,}")
print("\\nMotor de Redes Neuronales:  FUNCIONANDO CORRECTAMENTE")
print("=" * 80)
print("TODOS LOS EXPERIMENTOS COMPLETADOS EXITOSAMENTE")
print("=" * 80)
